In [1]:
import boto3
import json
from botocore.exceptions import ClientError

In [2]:
session = boto3.Session(profile_name='workshop')
config = session.client('config', region_name='us-west-2')

In [12]:
# All instances
query = """
SELECT *
WHERE resourceType = 'AWS::EC2::Instance'
"""

In [13]:
# Advanced queries
paginator = config.get_paginator('select_resource_config')
try:
    iterator = paginator.paginate(
        Expression=query,
        Limit=60,
    )
    for page in iterator:
        for resource in page['Results']:
            resource = json.loads(resource)
            print("Instance ID:", resource['resourceId'])
            print("Region:", resource['awsRegion'])
            print("arn:", resource['arn'])
            print("---------------------------------")
except ClientError as e:
    print(f"Error in region {region} with message: \n{e}")

Instance ID: i-0616c27f400776065
Region: us-west-2
arn: arn:aws:ec2:us-west-2:508595720710:instance/i-0616c27f400776065
---------------------------------
Instance ID: i-0f1cb3200020d87a7
Region: us-west-2
arn: arn:aws:ec2:us-west-2:508595720710:instance/i-0f1cb3200020d87a7
---------------------------------


In [32]:
query = """
SELECT
  resourceId,
  resourceType,
  supplementaryConfiguration.BucketVersioningConfiguration.status
WHERE
  resourceType = 'AWS::S3::Bucket'
  AND supplementaryConfiguration.BucketVersioningConfiguration.status != 'Enabled'
"""

In [33]:
paginator = config.get_paginator('select_resource_config')
try:
    iterator = paginator.paginate(
        Expression=query,
        Limit=60,
    )
    for page in iterator:
        for resource in page['Results']:
            resource = json.loads(resource)
            print("Bucket Name:", resource['resourceId'])
            print("Versioning:", resource['supplementaryConfiguration']['BucketVersioningConfiguration']['status'])
            print("---------------------------------")
except ClientError as e:
    print(f"Error in region {region} with message: \n{e}")

Bucket Name: cca-workshop-resources-workshops3bucket1-fv3hrwssaat3
Versioning: Off
---------------------------------
Bucket Name: cca-workshop-resources-workshops3bucket2-1tqh4wugpxnmc
Versioning: Off
---------------------------------
Bucket Name: cf-templates-br4e7sfl2kha-us-west-2
Versioning: Off
---------------------------------
Bucket Name: newdats-configbucket-yr1bo5siwwes
Versioning: Off
---------------------------------


In [39]:
# Create custom rule

rule_name = 's3-bucket-versioning-enabled-test'
description = 'Checks that S3 buckets have versioning enabled'
resource_type = 'AWS::S3::Bucket'
rule_content = """rule check_bucketversioning {
     supplementaryConfiguration.BucketVersioningConfiguration.status == "Enabled" <<
     result: NON_COMPLIANT
     message: S3 Bucket Versioning is NOT enabled.
     >>
}
"""

rule = {
    'ConfigRuleName': rule_name,
    'Description': description,
    'Scope': {
        'ComplianceResourceTypes': [
            resource_type,
        ],
    },
    'Source': {
        'Owner': 'CUSTOM_POLICY',
        'SourceDetails': [
            {
                'EventSource': 'aws.config',
                'MessageType': 'ConfigurationItemChangeNotification'
            },
        ],
        'CustomPolicyDetails': {
            'PolicyRuntime': 'guard-2.x.x',
            'PolicyText': rule_content,
            'EnableDebugLogDelivery': True,
        },
    },
}

response = config.put_config_rule(ConfigRule=rule)   


In [40]:
response

{'ResponseMetadata': {'RequestId': '13c584ad-9836-4c3c-8cd1-7aa47d9555f0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '13c584ad-9836-4c3c-8cd1-7aa47d9555f0',
   'strict-transport-security': 'max-age=86400',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 30 May 2023 20:00:24 GMT'},
  'RetryAttempts': 0}}

In [ ]:
# This rule checks if point in time recovery (PITR) is enabled on active Amazon DynamoDB tables
let status = ['ACTIVE']

rule tableisactive when
    resourceType == "AWS::DynamoDB::Table" {
    configuration.tableStatus == %status
}

rule checkcompliance when
    resourceType == "AWS::DynamoDB::Table"
    tableisactive {
        let pitr = supplementaryConfiguration.ContinuousBackupsDescription.pointInTimeRecoveryDescription.pointInTimeRecoveryStatus
        %pitr == "ENABLED"
}
                        